# Backpropagation Part 2

In [32]:
import numpy as np
from tensorflow.keras.layers import Softmax
from tensorflow.keras.losses import CategoricalCrossentropy

In [45]:
softmax = Softmax()


def my_softmax(y):
    z = np.sum(np.exp(y), axis=1, keepdims=True)
    log_probs = y - np.log(z)
    return np.exp(log_probs)

def my_better_softmax(y):
    shifted_logits = y - np.max(y, axis=1, keepdims=True)
    z = np.sum(np.exp(shifted_logits), axis=1, keepdims=True)
    log_probs = shifted_logits - np.log(z)
    return np.exp(log_probs)

In [46]:
y = np.array([[-200, 100, 5], [-200, 100, -5]]).astype(np.float64) * 1000
print(softmax(y))
print(my_softmax(y))
print(my_better_softmax(y))

tf.Tensor(
[[0. 1. 0.]
 [0. 1. 0.]], shape=(2, 3), dtype=float32)
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 1. 0.]
 [0. 1. 0.]]


/tmp/ipykernel_358000/838545724.py:5: RuntimeWarning: overflow encountered in exp
  z = np.sum(np.exp(y), axis=1, keepdims=True)


In [75]:
y_true = np.array([[0.3, 0.4, 0.3], [0.5, 0.5, 0.0], [1.0, 0.0, 0.0]])
y_pred = np.array([[0.3, 0.4, 0.29], [0.5, 0.5, 0.01], [0.99, 0.01, 0.01]])
cce = CategoricalCrossentropy()
cce(y_true, y_pred).numpy()

0.6040394277047364

In [76]:
def my_cross_entropy(y_true, y_pred):
    log_probs = np.log(y_pred)
    return -np.sum(y_true * log_probs) / y_true.shape[0]

my_cross_entropy(y_true, y_pred)

0.6007559857537915

In [80]:
np.identity(5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])